In [203]:
import numpy as np
import pandas as pd
import statsmodels.api as sm

In [204]:
df = pd.read_csv('data/train.csv')
df = df.rename({'Company ': 'Company'}, axis=1)
df = df.sort_values(['Company', 'ID'])
df = df.reset_index()

importantCols = ['SMA', 'EMA', 'WMA', 'DEMA', 'TEMA', 'TRIMA', 'KAMA', 'FAMA', 'MAMA', 'T3']
for col in importantCols:
    for i in range(1, 4):
        df.insert(df.shape[1], '{0}_{1}'.format(col, i), np.nan)
        
for index, row in df.iterrows():
    for col in importantCols:
        for i in range(1, 4):
            if index + i < df.shape[0] and df.at[index + 1, 'Company'] == row['Company']:
                df.at[index, '{0}_{1}'.format(col, i)] = df.at[index + i, col]

df = df.dropna()
df[:5]

,index,ID,Date,Company,SMA,EMA,WMA,DEMA,TEMA,TRIMA,...,KAMA_3,FAMA_1,FAMA_2,FAMA_3,MAMA_1,MAMA_2,MAMA_3,T3_1,T3_2,T3_3
54,162,163,03-04-2148,ABC,26.936,26.9160,27.1042,27.5128,27.2546,26.9737,...,25.6033,1.1623,1.2050,1.2482,9.5367,9.6995,9.8550,27.5508,27.5450,27.4724
55,165,166,06-04-2148,ABC,27.042,26.9313,27.1158,27.4320,27.1422,27.0740,...,25.5289,1.2050,1.2482,1.2920,9.6995,9.8550,10.0070,27.5450,27.4724,27.3319
56,168,169,07-04-2148,ABC,26.986,26.7274,26.8918,26.9703,26.5223,27.0930,...,25.5335,1.2482,1.2920,1.3364,9.8550,10.0070,10.1631,27.4724,27.3319,27.1490
57,171,172,08-04-2148,ABC,26.749,26.4588,26.5762,26.4377,25.8552,27.0033,...,25.5449,1.2920,1.3364,1.3813,10.0070,10.1631,10.3252,27.3319,27.1490,26.9623
58,174,175,09-04-2148,ABC,26.580,26.2045,26.2691,25.9792,25.3354,26.8447,...,25.5607,1.3364,1.3813,1.4269,10.1631,10.3252,10.4939,27.1490,26.9623,26.8113


In [205]:
corr = df[df.columns[3:]].corr().sort_values('Price', ascending=False)['Price']
corr

Price         1.000000
TEMA_2        0.999734
DEMA_3        0.999662
DEMA_2        0.999603
TEMA_3        0.999591
WMA_3         0.999499
TEMA_1        0.999445
EMA_3         0.999281
WMA_2         0.999244
DEMA_1        0.999218
EMA_2         0.999027
SMA_3         0.998983
TRIMA_3       0.998904
TEMA          0.998848
WMA_1         0.998789
KAMA_3        0.998670
DEMA          0.998630
SMA_2         0.998591
EMA_1         0.998582
TRIMA_2       0.998410
T3_3          0.998326
KAMA_2        0.998319
WMA           0.998212
SMA_1         0.998087
EMA           0.998017
MIDPRICE      0.997942
TRIMA_1       0.997857
T3_2          0.997821
KAMA_1        0.997816
MIDPOINT      0.997781
                ...   
SlowD         0.090743
PHASE         0.088746
FastD         0.084793
SlowK         0.084793
PPO           0.083061
AROONOSC      0.076934
FastK         0.071163
Aroon Up      0.070774
ADXR          0.064644
ADOSC         0.061550
CCI           0.059824
ADX           0.057177
MFI        

In [206]:
model_vars = [name for name in corr.index if corr[name] < 1.0 and corr[name] > 0.98]
model = sm.OLS(df['Price'], df[model_vars]).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.583e+06
Date:                Thu, 04 Jul 2019   Prob (F-statistic):               0.00
Time:                        03:43:00   Log-Likelihood:                -25826.
No. Observations:               11826   AIC:                         5.173e+04
Df Residuals:                   11788   BIC:                         5.201e+04
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
TEMA_2             123.1520    247.800      0.497      0.619    -362.577     608.881
DEMA_3              17.5773     14.783      1.189      0.234     -11.401      46.555
DEMA_2             -15.9846     13.087     -1.221      0.222     -41.637       9.668
TEMA_3            -245.2698    202.460     -1.211      0.226    -642.126     151.586
WMA_3             -479.5997    490.205     -0.978      0.328   -1440.482     481.282
TEMA_1             293.4570    261.322      1.123      0.261    -218.778     805.692
EMA_3             1088.8644    520.170      2.093      0.036      69.246    2108.483
WMA_2               63.4968    556.934      0.114      0.909   -1028.187    1155.180
DEMA_1              -0.0926      0.103     -0.897      0.370      -0.295       0.110
EMA_2             -377.4973    559.951     -0.674      0.500   -1475.094     720.099
SMA_3                0.8758      0.195      4.488      0.000       0.493       1.258
TRIMA_3              1.1202      0.809      1.384      0.166      -0.466       2.707
TEMA              -188.0585    135.510     -1.388      0.165    -453.681      77.564
WMA_1             -551.2184    556.000     -0.991      0.322   -1641.070     538.633
KAMA_3               0.0409      0.040      1.017      0.309      -0.038       0.120
DEMA                -0.2127      0.093     -2.289      0.022      -0.395      -0.031
SMA_2              -86.7815     89.130     -0.974      0.330    -261.491      87.928
EMA_1             -127.3347    549.124     -0.232      0.817   -1203.709     949.039
TRIMA_2             -0.0092      0.772     -0.012      0.991      -1.523       1.505
T3_3              -247.1348    209.010     -1.182      0.237    -656.828     162.559
KAMA_2               0.0720      0.072      0.994      0.320      -0.070       0.214
WMA                966.1428    492.910      1.960      0.050      -0.042    1932.328
SMA_1              -74.5659     97.834     -0.762      0.446    -266.337     117.205
EMA               -228.7809    420.427     -0.544      0.586   -1052.887     595.325
MIDPRICE            -0.0225      0.017     -1.337      0.181      -0.055       0.010
TRIMA_1             -1.0086      0.811     -1.243      0.214      -2.599       0.582
T3_2               604.6997    502.484      1.203      0.229    -380.252    1589.651
KAMA_1              -0.3323      0.073     -4.578      0.000      -0.475      -0.190
MIDPOINT             0.0293      0.022      1.338      0.181      -0.014       0.072
SMA               1645.8227   1077.811      1.527      0.127    -466.865    3758.510
Real Middle Band  1645.8227   1077.811      1.527      0.127    -466.865    3758.510
TRIMA               -0.3427      0.503     -0.682      0.495      -1.328       0.643
T3_1              -492.5970    403.107     -1.222      0.222   -1282.754     297.560
KAMA                 0.

In [207]:
abbreviated_names = ['SMA', 'EMA', 'WMA', 'DEMA', 'TEMA', 'TRIMA', 'KAMA', 'FAMA', 'MAMA', 'T3', 'MACD', 'MACD_Hist', 'MACD_Signal', 'MAC', 'MAC_Hist', 'MAC_Signal', 'SlowD', 'SlowK', 'FastD', 'FastK', 'RSI', 'FatD', 'FatK', 'WILLR', 'ADX', 'ADXR', 'APO', 'PPO', 'MOM', 'BOP', 'CCI', 'CMO', 'ROC', 'ROCR', 'Aroon Down', 'Aroon Up', 'AROONOSC', 'MFI', 'TRIX', 'ULTOSC', 'DX', 'MINUS_DI', 'PLUS_DI', 'MINUS_DM', 'PLUS_DM', 'Real Lower Band', 'Real Middle Band', 'Real Upper Band', 'MIDPOINT', 'MIDPRICE', 'SAR', 'TRANGE', 'ATR', 'NATR', 'Chaikin A/D', 'ADOSC', 'OBV', 'HT_TRENDLINE', 'LEAD SINE', 'SINE', 'TRENDMODE', 'DCPERIOD', 'HT_DCPHASE', 'PHASE', 'QUADRATURE']
full_names = ['simpleMovingAverage', 'exponentialMovingAverage', 'waysAndMeansAdvances', 'doubleExponentialMovingAverage', 'tripleExponentialMovingAverage', 'triangularMovingAverage', 'kaufmansAdaptiveMovingAverage', 'fAdaptiveMovingAverage', 'mesaAdaptiveMovingAverage', 't3MovingAverage', 'movingAverageConvergenceDivergence', 'movingAverageConvergenceDivergenceHistogram', 'movingAverageConvergenceDivergenceSignal', 'movingAverageConvergence', 'movingAverageConvergenceHistogram', 'movingAverageConvergenceSignal', 'slowStochasticOscillatorSecondLine', 'slowStochasticOscillatorMainLine', 'fastStochasticOscillatorSecondLine', 'fastStochasticOscillatorMainLine', 'relativeStrengthIndex', 'fastStochasticSecondLine', 'fastStochasticMainLine', 'willr', 'averageDirectionalIndex', 'averageDirectionalIndexRating', 'absolutePriceOscillator', 'percentagePriceOscillator', 'momentumIndicator', 'balanceOfPower', 'commodityChannelIndex', 'chandeMomentumOscillator', 'rateOfChange', 'rateOfChangeRating', 'aroonOscillatorDown', 'aroonOscillatorUp', 'aroonOscillator', 'moneyFlowIndex', 'tripleSmoothedExponentialMovingAverage', 'ultimateOscillatorDefinitionAndStrategies', 'directionalMovementIndex', 'minusDirectionalIndex', 'plusDirectionalIndex', 'minusDirectionalMovementIndex', 'plusDirectionalMovementIndex', 'relativeStrengthIndexLowerBand', 'relativeStrengthIndexMiddleBand', 'relativeStrengthIndexHighBand', 'midPoint', 'midPrice', 'parabolicStopAndReverse', 'trange', 'averageTrueRange', 'normalizedAverageTrueRange', 'chaikinAccumulationDistributionLine', 'accumulationDistributionOscillatorSC', 'onBalanceVolume', 'htTrendline', 'leadSine', 'sine', 'trendMode', 'dcPeriod', 'htDcPhase', 'phase', 'quadrature']

indexes = [int(name.split('_')[1]) for name in model.params.index if '_' in name and name not in abbreviated_names]

if len(indexes) > 0:
    print('if (nextRecords.size < {0}) {{'.format(max(indexes)))
    print('    return record.tripleExponentialMovingAverage')
    print('}')
    print()

print('return listOf(')
    
for name in model.params.index:
    modifier = model.params[name]
    comma = ',' if name != model.params.index[-1] else ''
    
    if '_' in name and name not in abbreviated_names:
        parts = name.split('_')
        index = int(parts[1]) - 1
        full_name = full_names[abbreviated_names.index(parts[0])]
        
        print('    nextRecords[{0}].{1} * {2}{3}'.format(index, full_name, modifier, comma))
    else:
        full_name = full_names[abbreviated_names.index(name)]
        
        print('    record.{0} * {1}{2}'.format(full_name, modifier, comma))

print(').sum()')

if (nextRecords.size < 3) {
    return record.tripleExponentialMovingAverage
}

return listOf(
    nextRecords[1].tripleExponentialMovingAverage * 123.15203358224193,
    nextRecords[2].doubleExponentialMovingAverage * 17.577344039041236,
    nextRecords[1].doubleExponentialMovingAverage * -15.984571190011554,
    nextRecords[2].tripleExponentialMovingAverage * -245.2698224864371,
    nextRecords[2].waysAndMeansAdvances * -479.5996636225168,
    nextRecords[0].tripleExponentialMovingAverage * 293.4570099420041,
    nextRecords[2].exponentialMovingAverage * 1088.864351708784,
    nextRecords[1].waysAndMeansAdvances * 63.49676268963094,
    nextRecords[0].doubleExponentialMovingAverage * -0.09261893252123932,
    nextRecords[1].exponentialMovingAverage * -377.4972663029023,
    nextRecords[2].simpleMovingAverage * 0.8758034381174127,
    nextRecords[2].triangularMovingAverage * 1.120192857895411,
    record.tripleExponentialMovingAverage * -188.0585459748954,
    nextRecords[0].waysAndMe